# Calculating Historical Free Cash Flows

In [1]:
import pandas as pd
import numpy as np

all_statements_path = 'Exxon Mobil Corporation NYSE XOM Financials.xls'

### Income Statement

In [2]:
inc_df = pd.read_excel(all_statements_path, sheet_name='Income Statement')

In [3]:
inc_df.head()

,Unnamed: 0,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
0,,NaN,NaN,NaN,NaN,NaN,NaN
1,Revenue,364763,239854,200628,237162,279332,260812
2,Other Revenue,-,1552,-,-,-,-
3,Total Revenue,364763,241406,200628,237162,279332,260812
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see this is a little messy. We can clean this up first by setting the index to be the first column. You can pass an integer index for a column to say that that column should be used as the index of the `DataFrame`. Here we want this first column, so we will pass `index_col=0` into `read_excel`.

In [4]:
inc_df = pd.read_excel(all_statements_path, sheet_name='Income Statement', index_col=0)

In [5]:
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,-,1552,-,-,-,-
Total Revenue,364763,241406,200628,237162,279332,260812
NaN,NaN,NaN,NaN,NaN,NaN,NaN


That's starting to look better. But we can see that there are some empty rows in the data. We can also see that there is a `-` in some values where there should be missing data. We want to remove the rows without any data, but first we want to fill in missing values when there is `-`, so that we can make sure a row of completely `-` would get removed. The missing representation in `pandas` is `NaN`, which we can specify manually through `numpy.nan`.

In [6]:
nc_df = inc_df.replace('-', np.nan)
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
,NaN,NaN,NaN,NaN,NaN,NaN
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,-,1552,-,-,-,-
Total Revenue,364763,241406,200628,237162,279332,260812
NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now we can see that those `-` got filled in for missing values. Now we want to remove the rows which have missing data. We can use `dropna` for this purpose. We must be careful to pass `how='all'` to `dropna` though. The default is `how='any'`, which means the row will be dropped if there is any missing value. Here we only want to drop the row if it is entirely missing, so we will use `how='all'`.

In [7]:
inc_df = inc_df.dropna(how='all')
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,-,1552,-,-,-,-
Total Revenue,364763,241406,200628,237162,279332,260812
Cost Of Goods Sold,234856,163605,132759,159053,190752,181228
Gross Profit,129907,77801,67869,78109,88580,79584


Now we can see that it has dropped the rows with all missing values, but not `Other Revenue` which has mostly missing values.

In [8]:
def load_and_clean_statement_df(statements_path, sheet_name):
    df = pd.read_excel(statements_path, sheet_name=sheet_name, index_col=0)
    df = df.replace('-', np.nan)
    df = df.dropna(how='all')
    return df

inc_df = load_and_clean_statement_df(all_statements_path, 'Income Statement')
inc_df.head()

,Dec-31-2014,Dec-31-2015,Dec-31-2016,Dec-31-2017,Dec-31-2018,Sep-30-2019
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,NaN,1552,NaN,NaN,NaN,NaN
Total Revenue,364763,241406,200628,237162,279332,260812
Cost Of Goods Sold,234856,163605,132759,159053,190752,181228
Gross Profit,129907,77801,67869,78109,88580,79584


### Balance Sheet

In [9]:
bs_df = load_and_clean_statement_df(all_statements_path, 'Balance Sheet')
bs_df.head()

,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-09-30
Cash And Equivalents,4616,3705,3657,3177,3042,5351
Total Cash & ST Investments,4616,3705,3657,3177,3042,5351
Accounts Receivable,18541,13243,16033,21274,19638,25308
Other Receivables,9468,6632,5361,4323,5063,NaN
Total Receivables,28009,19875,21394,25597,24701,25308


In [10]:
inc_df.columns = pd.to_datetime(inc_df.columns)
inc_df.head()

,2014-12-31,2015-12-31,2016-12-31,2017-12-31,2018-12-31,2019-09-30
Revenue,364763,239854,200628,237162,279332,260812
Other Revenue,NaN,1552,NaN,NaN,NaN,NaN
Total Revenue,364763,241406,200628,237162,279332,260812
Cost Of Goods Sold,234856,163605,132759,159053,190752,181228
Gross Profit,129907,77801,67869,78109,88580,79584


## Working with the Data - `pandas` Method

Now we want to calculate free cash flow. Here are the steps:
- Calculate non-cash expenses
- Calculate increase in working capital
- Calculate capital expenditures
- Calculate free cash flow from net income and the preceding items

### Calculate Non-Cash Expenses

For non-cash expenses, we need to total depreciation, amortization, stock-based compensation, impairment charges, and gains/losses on investments.

To see what we have in the two datasets, we can check the `.index` attribute of the `DataFrame`s.

In [11]:
inc_df.index

Index(['Revenue', 'Other Revenue', '  Total Revenue', 'Cost Of Goods Sold',
       '  Gross Profit', 'Selling General & Admin Exp.',
       'Exploration/Drilling Costs', 'Depreciation & Amort.',
       'Other Operating Expense/(Income)', '  Other Operating Exp., Total',
       '  Operating Income', 'Interest Expense', '  Net Interest Exp.',
       'Income/(Loss) from Affiliates', 'Currency Exchange Gains (Loss)',
       'Other Non-Operating Inc. (Exp.)', '  EBT Excl. Unusual Items',
       'Gain (Loss) On Sale Of Invest.', 'Gain (Loss) On Sale Of Assets',
       'Asset Writedown', '  EBT Incl. Unusual Items', 'Income Tax Expense',
       '  Earnings from Cont. Ops.', '  Net Income to Company',
       'Minority Int. in Earnings', '  Net Income',
       '  NI to Common Incl Extra Items', '  NI to Common Excl. Extra Items',
       'Basic EPS', 'Basic EPS Excl. Extra Items',
       'Weighted Avg. Basic Shares Out.', 'Diluted EPS',
       'Diluted EPS Excl. Extra Items', 'Weighted Avg. Dilu

Looking like we have depreciation, impairment (called asset writedown here) and gains/losses on investments and assets. There is no stock-based compensation so we will exclude that from the calculation.

To get the entire `Series` of data from the `DataFrame` by the `index` value, we can use `.loc` on the `DataFrame`.

In [12]:
inc_df.loc['Asset Writedown']

2014-12-31      NaN
2015-12-31      NaN
2016-12-31    -3600
2017-12-31    -2000
2018-12-31     -700
2019-09-30     -700
Name: Asset Writedown, dtype: object

That gives us each year's impairment. Notice that it's negative though, representing an expense. The calculation assumes it is positive. We can make it positive by using `abs` (absolute value). 

In [13]:
abs(inc_df.loc['Asset Writedown'])

2014-12-31     NaN
2015-12-31     NaN
2016-12-31    3600
2017-12-31    2000
2018-12-31     700
2019-09-30     700
Name: Asset Writedown, dtype: object

We can do math directly with `Series`.

In [14]:
inc_df.loc['Depreciation & Amort.']

2014-12-31    17297
2015-12-31    18048
2016-12-31    18708
2017-12-31    17893
2018-12-31    18045
2019-09-30    18403
Name: Depreciation & Amort., dtype: object

In [15]:
inc_df.loc['Depreciation & Amort.'] + abs(inc_df.loc['Asset Writedown'])

2014-12-31      NaN
2015-12-31      NaN
2016-12-31    22308
2017-12-31    19893
2018-12-31    18745
2019-09-30    19103
dtype: object

But what we notice is that if any value is missing, then it will cause the final calculation to be missing. This is not what we want. We want it to assume it is zero if it is missing. So let's fill the `DataFrame`s with zeroes if they are missing. We can do this using `fillna`.

In [16]:
inc_df = inc_df.fillna(0)
bs_df = bs_df.fillna(0)

In [17]:
abs(inc_df.loc['Asset Writedown'])

2014-12-31       0
2015-12-31       0
2016-12-31    3600
2017-12-31    2000
2018-12-31     700
2019-09-30     700
Name: Asset Writedown, dtype: object

In [18]:
inc_df.loc['Depreciation & Amort.'] + abs(inc_df.loc['Asset Writedown'])

2014-12-31    17297
2015-12-31    18048
2016-12-31    22308
2017-12-31    19893
2018-12-31    18745
2019-09-30    19103
dtype: object

In [19]:
non_cash_expenses = (
    inc_df.loc['Depreciation & Amort.'] + 
    abs(inc_df.loc['Asset Writedown']) + 
    inc_df.loc['Gain (Loss) On Sale Of Invest.'] + 
    inc_df.loc['Gain (Loss) On Sale Of Assets']
)  # NOTE: split onto multiple lines for readability, it would function exactly the same on one line without parentheses

In [20]:
non_cash_expenses

2014-12-31    20443
2015-12-31    18232
2016-12-31    23990
2017-12-31    20227
2018-12-31    20738
2019-09-30    21096
dtype: object

## Calculate Increase in Working Capital

First we need to calculate the net working capial, then calculate the change in that. Here we need balance sheet data, so let's look at the `index` there.

In [21]:
bs_df.index

Index(['Cash And Equivalents', '  Total Cash & ST Investments',
       'Accounts Receivable', 'Other Receivables', '  Total Receivables',
       'Inventory', 'Deferred Tax Assets, Curr.', 'Restricted Cash',
       'Other Current Assets', '  Total Current Assets',
       'Gross Property, Plant & Equipment', 'Accumulated Depreciation',
       '  Net Property, Plant & Equipment', 'Long-term Investments',
       'Deferred Tax Assets, LT', 'Other Long-Term Assets', 'Total Assets',
       'Accounts Payable', 'Accrued Exp.', 'Short-term Borrowings',
       'Curr. Port. of LT Debt', 'Curr. Port. of Cap. Leases',
       'Curr. Income Taxes Payable', 'Other Current Liabilities',
       '  Total Current Liabilities', 'Long-Term Debt', 'Capital Leases',
       'Pension & Other Post-Retire. Benefits',
       'Def. Tax Liability, Non-Curr.', 'Other Non-Current Liabilities',
       'Total Liabilities', 'Common Stock', 'Retained Earnings',
       'Treasury Stock', 'Comprehensive Inc. and Other',
     

In [22]:
nwc = bs_df.loc['Accounts Receivable'] + bs_df.loc['Inventory'] - bs_df.loc['Accounts Payable']

In [23]:
nwc

2014-12-31     9933
2015-12-31    11414
2016-12-31    13312
2017-12-31    16565
2018-12-31    17533
2019-09-30     3562
dtype: object

To get a change, we can take advantage of `Series.shift`, which shifts all the values by a number of rows.

In [24]:
nwc.shift(1)

2014-12-31      NaN
2015-12-31     9933
2016-12-31    11414
2017-12-31    13312
2018-12-31    16565
2019-09-30    17533
dtype: object

We can see that with `shift(1)`, the value from last period is now in this period. So `nwc.shift(1)` is representing last year's net working capital. So then the change in net working capital is simply:

In [25]:
change_nwc = nwc - nwc.shift(1)

In [26]:
change_nwc

2014-12-31       NaN
2015-12-31      1481
2016-12-31      1898
2017-12-31      3253
2018-12-31       968
2019-09-30    -13971
dtype: object

### Calculate Capital Expenditures

Here we need to first get the change in net property, plant, and equipment, then add the current depreciation to get capital expenditures.

In [27]:
bs_df.loc['  Net Property, Plant & Equipment']

2014-12-31    252668
2015-12-31    251605
2016-12-31    244224
2017-12-31    252630
2018-12-31    247101
2019-09-30    257065
Name:   Net Property, Plant & Equipment, dtype: object

In [28]:
change_ppe = bs_df.loc['  Net Property, Plant & Equipment'] - bs_df.loc['  Net Property, Plant & Equipment'].shift(1)

In [29]:
change_ppe

2014-12-31      NaN
2015-12-31    -1063
2016-12-31    -7381
2017-12-31     8406
2018-12-31    -5529
2019-09-30     9964
Name:   Net Property, Plant & Equipment, dtype: object

In [30]:
capex = change_ppe + inc_df.loc['Depreciation & Amort.']

In [31]:
capex

2014-12-31      NaN
2015-12-31    16985
2016-12-31    11327
2017-12-31    26299
2018-12-31    12516
2019-09-30    28367
dtype: object

### Put it All Together into FCFs

In [32]:
fcf = inc_df.loc['  Net Income'] + non_cash_expenses - change_nwc - capex

In [33]:
fcf

2014-12-31      NaN
2015-12-31    15916
2016-12-31    18605
2017-12-31    10385
2018-12-31    28094
2019-09-30    21350
dtype: object

In [34]:
!pip install finstmt

### Using `finstmt`

I have created three main classes (so far) to help working with the data. `IncomeStatements`, `BalanceSheets`, and `FinancialStatements`. Let's import them.

In [35]:
from finstmt import IncomeStatements, BalanceSheets, FinancialStatements

In [36]:
inc_df = load_and_clean_statement_df(all_statements_path, 'Income Statement')
bs_df = load_and_clean_statement_df(all_statements_path, 'Balance Sheet')

Now we want to create `IncomeStatements` from the income statements and `BalanceSheets` from the balance sheets. The way to do this is using `.from_df` methods of each.

In [37]:
inc_data = IncomeStatements.from_df(inc_df)

INFO: Was not able to extract data from the following names: {'Income/(Loss) from Affiliates', 'EBIT', 'Normalized Basic EPS', '  Net Interest Exp.', 'Weighted Avg. Basic Shares Out.', '  EBT Excl. Unusual Items', '  Gross Profit', '  Other Operating Exp., Total', 'Other Revenue', 'Other Non-Operating Inc. (Exp.)', '  Net Income to Company', 'Restatement Type', 'Calculation Type', 'EBITA', '  Earnings from Cont. Ops.', 'EBITDAR', 'Weighted Avg. Diluted Shares Out.', '  NI to Common Incl Extra Items', 'Filing Date', 'Basic EPS Excl. Extra Items', 'Normalized Diluted EPS', 'Normalized Net Income', 'Minority Int. in Earnings', 'Diluted EPS', 'Payout Ratio %', 'Effective Tax Rate %', 'Shares per Depository Receipt', '  Stock-Based Comp., Total', '  Total Revenue', 'Interest Capitalized', 'Currency Exchange Gains (Loss)', 'Diluted EPS Excl. Extra Items', 'Dividends per Share', 'Stock-Based Comp., Unallocated', 'Basic EPS', '  NI to Common Excl. Extra Items', 'As Reported Total Revenue*', 'N

C:\Users\DELL\anaconda3\lib\site-packages\finstmt\findata\period_data.py:116: UserWarning: Previously had revenue extracted from "Revenue". Replacing with value from "  Total Revenue"
  warnings.warn(


The first feature that we get from this package is much cleaner output out of the box. Simply display the data and it has cleaned up all the names of the variables, standardized the date format, and formatted values in currency format.

In [38]:
inc_data

,12/31/2014,12/31/2015,12/31/2016,12/31/2017,12/31/2018,09/30/2019
Revenue,"$364,763","$241,406","$200,628","$237,162","$279,332","$260,812"
Cost of Goods Sold,"$234,856","$163,605","$132,759","$159,053","$190,752","$181,228"
Gross Profit,"$129,907","$77,801","$67,869","$78,109","$88,580","$79,584"
SG&A Expense,"$12,002","$10,961","$11,783","$11,893","$12,300","$12,094"
Interest Expense,$286,$311,$453,$601,$766,$844
Income Tax Expense,"$18,015","$5,415",$-406,"$-1,174","$9,532","$6,513"
R&D Expense,-,-,-,-,-,-
Depreciation & Amortization Expense,"$17,297","$18,048","$18,708","$17,893","$18,045","$18,403"
Other Operating Expenses,"$64,857","$32,834","$31,375","$32,459","$35,230","$33,161"
Gain on Sale of Investments,$-5,$-42,-,-,-,-


In [41]:
inc_data.revenue

2014-12-31    364763.0
2015-12-31    241406.0
2016-12-31    200628.0
2017-12-31    237162.0
2018-12-31    279332.0
2019-09-30    260812.0
Name: Revenue, dtype: float64

You can also see we get back to the raw numbers when accessing a variable, even though when we show the full statement, it is formatted.

We can also pull out one or more dates from the statements easily. Notice also that I'm not even using the same format of the dates, it understands that what you are passing is a date and converts it to match the column.

In [42]:
inc_data['2014-12-31']

,0
Revenue,"$364,763"
Cost of Goods Sold,"$234,856"
Gross Profit,"$129,907"
SG&A Expense,"$12,002"
Interest Expense,$286
Income Tax Expense,"$18,015"
R&D Expense,-
Depreciation & Amortization Expense,"$17,297"
Other Operating Expenses,"$64,857"
Gain on Sale of Investments,$-5


In [43]:
inc_data[['2014-12-31', '2015-12-31']]

,12/31/2014,12/31/2015
Revenue,"$364,763","$241,406"
Cost of Goods Sold,"$234,856","$163,605"
Gross Profit,"$129,907","$77,801"
SG&A Expense,"$12,002","$10,961"
Interest Expense,$286,$311
Income Tax Expense,"$18,015","$5,415"
R&D Expense,-,-
Depreciation & Amortization Expense,-,-
Other Operating Expenses,"$64,857","$32,834"
Gain on Sale of Investments,$-5,$-42


Now let's go ahead and construct the balance sheet.

In [44]:
bs_data = BalanceSheets.from_df(bs_df)

INFO: Was not able to extract data from the following names: {'Inventory Method', 'Total Shares Out. on Filing Date', 'Debt Equiv. of Unfunded Proj. Benefit Obligation', 'LIFO Reserve', 'Curr. Port. of Cap. Leases', 'Tangible Book Value', '  Total Common Equity', 'Full Time Employees', 'Restricted Cash', 'Accum. Allowance for Doubtful Accts', 'Treasury Stock', 'Restatement Type', 'Book Value/Share', 'Finished Goods Inventory', 'Curr. Income Taxes Payable', 'Total Shares Out. on Balance Sheet Date', 'Net Debt', 'Filing Date', 'Total Minority Interest', 'Accrued Exp.', 'Other Receivables', 'Tangible Book Value/Share', 'Equity Method Investments', '  Total Receivables', 'Raw Materials Inventory', 'Pension & Other Post-Retire. Benefits', 'Capital Leases', 'Debt Equivalent Oper. Leases', 'Calculation Type'}


In [45]:
bs_data

,12/31/2014,12/31/2015,12/31/2016,12/31/2017,12/31/2018,09/30/2019
Cash and Cash Equivalents,"$4,616","$3,705","$3,657","$3,177","$3,042","$5,351"
Short-Term Investments,-,-,-,-,-,-
Cash and Short-Term Investments,"$4,616","$3,705","$3,657","$3,177","$3,042","$5,351"
Receivables,"$18,541","$13,243","$16,033","$21,274","$19,638","$25,308"
Inventory,"$16,678","$16,245","$15,080","$16,992","$18,958","$17,590"
"Deferred Tax Assets, Current","$2,001","$1,329",-,-,-,-
Other Current Assets,"$1,564","$1,469","$1,285","$1,368","$1,272","$1,759"
Total Current Assets,"$52,910","$42,623","$41,416","$47,134","$47,973","$50,008"
"Gross Property, Plant & Equipment","$446,789","$447,337","$453,915","$477,185","$477,190",-
Accumulated Depreciation,"$194,121","$195,732","$209,691","$224,555","$230,089",-


In [46]:
stmts = FinancialStatements(inc_data, bs_data)

In [47]:
stmts

,12/31/2014,12/31/2015,12/31/2016,12/31/2017,12/31/2018,09/30/2019
Revenue,"$364,763","$241,406","$200,628","$237,162","$279,332","$260,812"
Cost of Goods Sold,"$234,856","$163,605","$132,759","$159,053","$190,752","$181,228"
Gross Profit,"$129,907","$77,801","$67,869","$78,109","$88,580","$79,584"
SG&A Expense,"$12,002","$10,961","$11,783","$11,893","$12,300","$12,094"
Interest Expense,$286,$311,$453,$601,$766,$844
Income Tax Expense,"$18,015","$5,415",$-406,"$-1,174","$9,532","$6,513"
R&D Expense,-,-,-,-,-,-
Depreciation & Amortization Expense,"$17,297","$18,048","$18,708","$17,893","$18,045","$18,403"
Other Operating Expenses,"$64,857","$32,834","$31,375","$32,459","$35,230","$33,161"
Gain on Sale of Investments,$-5,$-42,-,-,-,-


In [48]:
stmts.cash

2014-12-31    4616.0
2015-12-31    3705.0
2016-12-31    3657.0
2017-12-31    3177.0
2018-12-31    3042.0
2019-09-30    5351.0
Name: Cash and Cash Equivalents, dtype: float64

In [49]:
stmts.cogs

2014-12-31    234856.0
2015-12-31    163605.0
2016-12-31    132759.0
2017-12-31    159053.0
2018-12-31    190752.0
2019-09-30    181228.0
Name: Cost of Goods Sold, dtype: float64

In [50]:
stmts['2014-12-31']

,12/31/2014
Revenue,"$364,763"
Cost of Goods Sold,"$234,856"
Gross Profit,"$129,907"
SG&A Expense,"$12,002"
Interest Expense,$286
Income Tax Expense,"$18,015"
R&D Expense,-
Depreciation & Amortization Expense,"$17,297"
Other Operating Expenses,"$64,857"
Gain on Sale of Investments,$-5


In [51]:
stmts[['2014-12-31', '2015-12-31']]

,12/31/2014,12/31/2015
Revenue,"$364,763","$241,406"
Cost of Goods Sold,"$234,856","$163,605"
Gross Profit,"$129,907","$77,801"
SG&A Expense,"$12,002","$10,961"
Interest Expense,$286,$311
Income Tax Expense,"$18,015","$5,415"
R&D Expense,-,-
Depreciation & Amortization Expense,-,-
Other Operating Expenses,"$64,857","$32,834"
Gain on Sale of Investments,$-5,$-42


In [52]:
stmts[['2014-12-31', '2015-12-31']].total_equity

2014-12-31    181064.0
2015-12-31    176810.0
Name: Total Stockholder's Equity, dtype: float64

### Calculating FCFs Using `finstmt`

In [53]:
stmts.fcf

2014-12-31        NaN
2015-12-31    15916.0
2016-12-31    18605.0
2017-12-31    10385.0
2018-12-31    28094.0
2019-09-30    21350.0
dtype: float64

In [54]:
stmts.net_ppe

2014-12-31    252668.0
2015-12-31    251605.0
2016-12-31    244224.0
2017-12-31    252630.0
2018-12-31    247101.0
2019-09-30    257065.0
Name: Net Property, Plant & Equipment, dtype: float64

In [55]:
stmts.lag('net_ppe', 1)

2014-12-31         NaN
2015-12-31    252668.0
2016-12-31    251605.0
2017-12-31    244224.0
2018-12-31    252630.0
2019-09-30    247101.0
Name: Net Property, Plant & Equipment, dtype: float64

In [56]:
stmts.change('net_ppe')

2014-12-31       NaN
2015-12-31   -1063.0
2016-12-31   -7381.0
2017-12-31    8406.0
2018-12-31   -5529.0
2019-09-30    9964.0
Name: Net Property, Plant & Equipment, dtype: float64

In [57]:
stmts.net_income + stmts.non_cash_expenses - stmts.change('nwc') - stmts.capex

2014-12-31        NaN
2015-12-31    15916.0
2016-12-31    18605.0
2017-12-31    10385.0
2018-12-31    28094.0
2019-09-30    21350.0
dtype: float64